In [ ]:
if 'google.colab' in str(get_ipython()):
  !pip install -q condacolab
  import condacolab
  condacolab.install()

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
BRANCH = 'main'
if 'google.colab' in str(get_ipython()):
  !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

In [ ]:
if 'google.colab' in str(get_ipython()):
  ! conda install -c conda-forge pynini=2.1.3
  ! mkdir images
  ! wget https://github.com/NVIDIA/NeMo/blob/$BRANCH/tutorials/text_processing/images/deployment.png -O images/deployment.png
  ! wget https://github.com/NVIDIA/NeMo/blob/$BRANCH/tutorials/text_processing/images/pipeline.png -O images/pipeline.png

In [ ]:
import os
import wget
import pynini
import nemo_text_processing


# Task Description

Text normalization (TN) is a part of the Text-To-Speech (TTS) pre-processing pipeline. It could also be used for pre-processing Automatic Speech Recognition (ASR) training transcripts.

TN is the task of converting text in written form to its spoken form to improve TTS. For example, `10:00` should be changed to `ten o'clock` and `10kg` to `ten kilograms`.

# NeMo Text Normalization

NeMo TN is based on weighted finite-state
transducer (WFST) grammars. The tool uses [`Pynini`](https://github.com/kylebgorman/pynini) to construct WFSTs, and the created grammars can be exported and integrated into [`Sparrowhawk`](https://github.com/google/sparrowhawk) (an open-source version of [The Kestrel TTS text normalization system](https://www.cambridge.org/core/journals/natural-language-engineering/article/abs/kestrel-tts-text-normalization-system/F0C18A3F596B75D83B75C479E23795DA)) for production. The NeMo TN tool can be seen as a Python extension of `Sparrowhawk`. 

Currently, NeMo TN provides support for English and the following semiotic classes from the [Google Text normalization dataset](https://www.kaggle.com/richardwilliamsproat/text-normalization-for-english-russian-and-polish):
DATE, CARDINAL, MEASURE, DECIMAL, ORDINAL, MONEY, TIME, TELEPHONE, ELECTRONIC, PLAIN. We additionally added the class `WHITELIST` for all whitelisted tokens whose verbalizations are directly looked up from a user-defined list.

The toolkit is modular, easily extendable, and can be adapted to other languages and tasks like [inverse text normalization](https://github.com/NVIDIA/NeMo/blob/stable/tutorials/text_processing/Inverse_Text_Normalization.ipynb). The Python environment enables an easy combination of text covering grammars with NNs. 

The rule-based system is divided into a classifier and a verbalizer following  [Google's Kestrel](https://www.researchgate.net/profile/Richard_Sproat/publication/277932107_The_Kestrel_TTS_text_normalization_system/links/57308b1108aeaae23f5cc8c4/The-Kestrel-TTS-text-normalization-system.pdf) design: the classifier is responsible for detecting and classifying semiotic classes in the underlying text, the verbalizer the verbalizes the detected text segment. 
In the example `The alarm goes off at 10:30 a.m.`, the tagger for TIME detects `10:30 a.m.` as a valid time data with `hour=10`, `minutes=30`, `suffix=a.m.`, the verbalizer then turns this into `ten thirty a m`.

The overall NeMo TN pipeline from development in `Pynini` to deployment in `Sparrowhawk` is shown below (example for ITN):
![alt text](images/deployment.png "Inverse Text Normalization Pipeline")



# Quick Start

## Add TN to your Python TTS pre-processing workflow

TN is a part of the `nemo_text_processing` package which is installed with `nemo_toolkit`. Installation instructions could be found [here](https://github.com/NVIDIA/NeMo/tree/main/README.rst).

In [ ]:
from nemo_text_processing.text_normalization.normalize import Normalizer
# creates normalizer object that works on lower cased input
normalizer = Normalizer(input_case='cased', lang='en')
raw_text = "We paid $123 for this desk."
normalizer.normalize(raw_text, verbose=False)

In the above cell, `$123` would be converted to `one hundred twenty three dollars`, and the rest of the words remain the same.

## Run Text Normalization on an input from a file

Use `run_predict.py` to convert a written format from a file `INPUT_FILE` to a spoken text and save the output to `OUTPUT_FILE`. Under the hood, `run_predict.py` is calling `normalize()` (see the above section).

In [ ]:
# If you're running the notebook locally, update the NEMO_TEXT_PROCESSING_PATH below
# In Colab, a few required scripts will be downloaded from NeMo github

NEMO_TOOLS_PATH = '<UPDATE_PATH_TO_NeMo_root>/nemo_text_processing/text_normalization'
DATA_DIR = 'data_dir'
os.makedirs(DATA_DIR, exist_ok=True)

if 'google.colab' in str(get_ipython()):
    NEMO_TOOLS_PATH = '.'

    required_files = ['run_predict.py',
                      'run_evaluate.py']
    for file in required_files:
        if not os.path.exists(file):
            file_path = 'https://raw.githubusercontent.com/NVIDIA/NeMo/' + BRANCH + '/nemo_text_processing/text_normalization/' + file
            print(file_path)
            wget.download(file_path)
elif not os.path.exists(NEMO_TOOLS_PATH):
      raise ValueError(f'update path to NeMo root directory')

In [ ]:
INPUT_FILE = f'{DATA_DIR}/test.txt'
OUTPUT_FILE = f'{DATA_DIR}/test_tn.txt'

! echo "The alarm went off at 10:00." > $DATA_DIR/test.txt
! cat $INPUT_FILE
! python $NEMO_TOOLS_PATH/run_predict.py --input=$INPUT_FILE --output=$OUTPUT_FILE --language='en'

In [ ]:
# check that the raw text was converted to the spoken form
! cat $OUTPUT_FILE

## Run evaluation

[Google Text normalization dataset](https://www.kaggle.com/richardwilliamsproat/text-normalization-for-english-russian-and-polish) consists of 1.1 billion words of English text from Wikipedia, divided across 100 files. The normalized text is obtained with The Kestrel TTS text normalization system).

To run evaluation, the input file should follow the Google Text normalization dataset format. That is, every line of the file needs to have the format `<semiotic class>\t<unnormalized text>\t<self>` if it's trivial class or `<semiotic class>\t<unnormalized text>\t<normalized text>` in case of a semiotic class.


Example evaluation run:


`python run_evaluate.py \
        --input=./en_with_types/output-00001-of-00100 \
        [--language LANGUAGE] \
        [--input_case INPUT_CASE] \
        [--cat CATEGORY]`

Use `--cat` to specify a `CATEGORY` to run evaluation on (all other categories are going to be excluded from evaluation). The option `--input_case` tells the algorithm that the input is either lower cased or cased.





In [ ]:
eval_text =  """PLAIN\ton\t<self>
DATE\t22 july 2012\tthe twenty second of july twenty twelve
PLAIN\tthey\t<self>
PLAIN\tworked\t<self>
PLAIN\tuntil\t<self>
TIME\t12:00\ttwelve o'clock
<eos>\t<eos>
"""
INPUT_FILE_EVAL = f"{DATA_DIR}/test_eval.txt"
with open(INPUT_FILE_EVAL, 'w') as fp:
  fp.write(eval_text)
! cat $INPUT_FILE_EVAL

In [ ]:
! python $NEMO_TOOLS_PATH/run_evaluate.py --input=$INPUT_FILE_EVAL --language='en'

`run_evaluate.py` call will output both **sentence level** and **token level** accuracies. 
For our example, the expected output is the following:

```
Loading training data: data_dir/test_eval.txt
Sentence level evaluation...
- Data: 1 sentences
100% 1/1 [00:00<00:00, 14.24it/s]
- Normalized. Evaluating...
- Accuracy: 1.0
Token level evaluation...
- Token type: PLAIN
  - Data: 4 tokens
100% 4/4 [00:00<00:00, 239.56it/s]
  - Denormalized. Evaluating...
  - Accuracy: 1.0
- Token type: DATE
  - Data: 1 tokens
100% 1/1 [00:00<00:00, 33.69it/s]
  - Denormalized. Evaluating...
  - Accuracy: 1.0
- Token type: TIME
  - Data: 1 tokens
100% 1/1 [00:00<00:00, 94.84it/s]
  - Denormalized. Evaluating...
  - Accuracy: 1.0
- Accuracy: 1.0
 - Total: 6 

 - Total: 6 

Class      | Num Tokens | Normalization
sent level | 1          | 1.0  
PLAIN      | 4          | 1.0  
DATE       | 1          | 1.0  
CARDINAL   | 0          | 0    
LETTERS    | 0          | 0    
VERBATIM   | 0          | 0    
MEASURE    | 0          | 0    
DECIMAL    | 0          | 0    
ORDINAL    | 0          | 0    
DIGIT      | 0          | 0    
MONEY      | 0          | 0    
TELEPHONE  | 0          | 0    
ELECTRONIC | 0          | 0    
FRACTION   | 0          | 0    
TIME       | 1          | 1.0  
ADDRESS    | 0          | 0 

```


# C++ deployment

The instructions on how to export `Pynini` grammars and to run them with `Sparrowhawk`, could be found at [NeMo/tools/text_processing_deployment](https://github.com/NVIDIA/NeMo/tree/main/tools/text_processing_deployment).

# WFST and Common Pynini Operations

See [NeMo Text Inverse Normalization Tutorial](https://github.com/NVIDIA/NeMo/blob/stable/tutorials/text_processing/Inverse_Text_Normalization.ipynb) for details.

# References and Further Reading:


- [Zhang, Yang, Bakhturina, Evelina, Gorman, Kyle and Ginsburg, Boris. "NeMo Inverse Text Normalization: From Development To Production." (2021)](https://arxiv.org/abs/2104.05055)
- [Ebden, Peter, and Richard Sproat. "The Kestrel TTS text normalization system." Natural Language Engineering 21.3 (2015): 333.](https://www.cambridge.org/core/journals/natural-language-engineering/article/abs/kestrel-tts-text-normalization-system/F0C18A3F596B75D83B75C479E23795DA)
- [Gorman, Kyle. "Pynini: A Python library for weighted finite-state grammar compilation." Proceedings of the SIGFSM Workshop on Statistical NLP and Weighted Automata. 2016.](https://www.aclweb.org/anthology/W16-2409.pdf)
- [Mohri, Mehryar, Fernando Pereira, and Michael Riley. "Weighted finite-state transducers in speech recognition." Computer Speech & Language 16.1 (2002): 69-88.](https://cs.nyu.edu/~mohri/postscript/csl01.pdf)